In [ ]:
!sudo dnf install gcc-c++ -y

Updating Subscription Management repositories.
Unable to read consumer identity
Subscription Manager is operating in container mode.

This system is not registered with an entitlement server. You can use subscription-manager to register.

Red Hat Universal Base Image 9 (RPMs) - BaseOS  6.8 kB/s | 3.8 kB     00:00    
Red Hat Universal Base Image 9 (RPMs) - BaseOS   76 kB/s | 514 kB     00:06    
Red Hat Universal Base Image 9 (RPMs) - AppStre  22 kB/s | 4.2 kB     00:00    
Red Hat Universal Base Image 9 (RPMs) - AppStre 303 kB/s | 1.8 MB     00:06    
Red Hat Universal Base Image 9 (RPMs) - CodeRea  12 kB/s | 4.2 kB     00:00    


In [ ]:
!sudo dnf install cmake -y

In [1]:
!pip install --upgrade pip

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.8 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip config unset global.target

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Writing to /home/mosaic-ai/.config/pip/pip.conf


In [1]:
!pip install llama-cpp-python

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 156.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 178.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      *** scikit-build-core 0.6.1 using CMake 3.27.7 (wheel)
      *** Configuring CMake...
      loading initial cache file /tmp/tmpwbpmjn9c/build/CMakeInit.txt
      -- The C compiler identification is GNU 11.3.1
      -- The CXX compiler identification is unknown
      -- Detecting C compiler ABI

In [2]:
from llama_cpp import Llama

In [12]:
llm = Llama(model_path="/data/mistral/query-to-mql/exp-10/nov-20/merged-model/ggml-model-q4_0/ggml-model-q4_0.gguf", n_threads=8)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 1 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [13]:
query_template_v1 = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

context = """{
    "MEASURE": [{"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]}],
    "FILTER": [{"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date"]}]
    }"""

date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [14]:
user_query = 'sales in nov 2020 to may 2021 vs Q3 2021'

inp = query_template_v1.format(context=context,
                                   user_query=user_query,
                                  date_input=date_input)

In [16]:
import time

In [29]:
start = time.time()
output = llm(
  inp, # Prompt
  max_tokens=200, # Generate up to 32 tokens
  echo=True, # Echo the prompt back in the output
    temperature=0
)
print("time taken : ", time.time()-start)

Llama.generate: prefix-match hit


time taken :  20.7693088054657


In [28]:
print(output['choices'][0]['text'].split('[MQL]\n')[1].split('\n[/MQL]')[0])

{'DATE VARIABLE': {'nov 2020 to may 2021': [{'CONVERTED TIME ELEMENT': 'nov 2020 to may 2021', 'DATE RANGE': '2020/11/01 - 2021/05/31', 'ENTITY': 'Order Date'}]}, 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}


#### Steps
1. Install c++ compile
    sudo dnf install gcc-c++ -y
2. Install cmake
    sudo dnf install cmake -y
3. clone and build llama.cpp from - https://github.com/ggerganov/llama.cpp

4. Install llama.cpp python client
    a. pip install --upgrade pip
    b. remove any content from pip.conf/pip.ini as it creates issue during building wheel for llama-cpp-python.
    c. pip install llama-cpp-python